In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [2]:
cd MyDrive/MyDrive/프로젝트/한영데이터

/content/MyDrive/MyDrive/프로젝트/한영데이터


In [ ]:
! pip install sentencepiece
! pip install transformers

In [23]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from collections import Counter
import pandas as pd
from tqdm import tqdm
import torch.nn as nn
import pickle
import os

# Data 통합

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import pickle

In [ ]:
data_list = os.listdir()

In [ ]:
print(data_list[0])

3_문어체_뉴스(1)_200226.xlsx


In [ ]:
한글 = []
영어 = []

In [ ]:
for i in data_list:
    data = pd.read_excel(i,header=0)
    한글.extend(data['원문'].tolist())
    영어.extend(data['번역문'].tolist())

In [ ]:
Data = pd.DataFrame(columns=['원문','번역문'])
Data['한글']=한글
Data['번역문']=영어

In [ ]:
Data.to_csv('./train_data.csv',encoding='utf-8')

# Data 불러오기

In [7]:
tokenizer = T5Tokenizer.from_pretrained("google/mt5-base")
model = T5ForConditionalGeneration.from_pretrained("google/mt5-base")
data = pd.read_csv('./train_data.csv',header=0,index_col=0, encoding='utf-8')

Downloading:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Downloading:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## Tokenizer에 대한 연구, Counter

In [8]:
cnt_kor = Counter()
cnt_en = Counter()
for i in tqdm(data['원문']):
    cnt_kor.update(tokenizer.encode(i))
for i in tqdm(data['번역문']):
    cnt_en.update(tokenizer.encode(i))
ㅇ
# kor의 token이 얼마나 활용됬는 지를 파악 : 11%
print('korean')
print(len(cnt_kor.keys())/len(tokenizer.get_vocab()))
# token의 개수당, 전체 빈도 비율
for i in [10000,20000,len(cnt_kor)]:
    print(sum([j[1] for j in cnt_kor.most_common(i)])/sum(cnt_kor.values())) # 20000만 정도면 - 99.9% 카바
# 영어의 token이 전체에서 차지하는 비중 : 22%
print('englishg')
print(len(cnt_en.keys())/len(tokenizer.get_vocab()))
for i in [10000,20000,30000,len(cnt_en)]: # 30000만 - 99.8% 카바
    print(sum([j[1] for j in cnt_en.most_common(i)])/sum(cnt_en.values()))    

100%|██████████| 1001042/1001042 [04:01<00:00, 4150.91it/s]


korean
0.11483406637345062
0.9990416393851086
0.9997537741731923
1.0
englishg
0.2183046781287485
0.9809282187582283
0.9948252063664275
0.9981891432585646
1.0


In [ ]:
# byte 들이 존재함.
for _,i in enumerate(tokenizer.get_vocab()):
    if _<300:  # 259까지 functional tokens
        print(i)

In [16]:
# token - 한국어 20000개 + 영어 30000개 + 100개 special tokens 
new_tokens = set()
for k,v in cnt_kor.most_common(20000):
    new_tokens.add(k)
for k,v in cnt_en.most_common(30000):
    new_tokens.add(k)
for k in range(259):
    new_tokens.add(k)

In [17]:
print(len(new_tokens)) # 40426

40426


In [ ]:
# 맨 마지막 100개가 extra_id -> 100개 ; t5에서 추가했던 부분들 -> 이 부분은 load할 떄 하면 된다.

In [19]:
tokens = sorted(new_tokens)

In [20]:
len(tokens) # 40320

40426

# Updating the model  


In [24]:
new_emb_size = len(tokens) # 
new_emb = nn.Embedding(new_emb_size, model.shared.embedding_dim,padding_idx=tokenizer.pad_token_id)
new_head = nn.Linear(model.lm_head.in_features, new_emb_size, bias=False)
# weight copy
for new_id, old_id in enumerate(tokens):
    new_emb.weight.data[new_id]=model.shared.weight.data[old_id]
    new_head.weight.data[new_id]=model.lm_head.weight.data[old_id]
model.shared.weight = new_emb.weight
model.lm_head.weight = new_head.weight
model.config.__dict__['vocab_size']=new_emb_size
model.config.__dict__['_name_or_path']='cointergrated/kor-base'
model.save_pretrained('kor-base')

In [25]:
print(model.shared.weight.shape)
print(model.lm_head.weight.shape)

torch.Size([40426, 768])
torch.Size([40426, 768])


## Update Sentencepiece  
C로 제작되어 있어서 다른 방식으로 copycopy

# 기존 tokenizer vocabulary save

In [27]:
tokenizer.save_pretrained('/content/MyDrive/MyDrive/프로젝트/한영데이터/mt5')  # 저장되는 것이 다르넹..

('/content/MyDrive/MyDrive/프로젝트/한영데이터/mt5/tokenizer_config.json',
 '/content/MyDrive/MyDrive/프로젝트/한영데이터/mt5/special_tokens_map.json',
 '/content/MyDrive/MyDrive/프로젝트/한영데이터/mt5/spiece.model',
 '/content/MyDrive/MyDrive/프로젝트/한영데이터/mt5/added_tokens.json')

## SentencePiece (Tokenizer) Customize  
sentencepiece 설치 후, sentencepiece_model_pb2를 부름.  
그 다음에 원래 tokenizer.save_vocabulary('mt5-base')하면, sentencepiece의 piece와 score가 저장된다. 이를 불러온다.  


In [34]:
from sentencepiece import sentencepiece_model_pb2 as model
# ModelProto 그리고, 기존에 저장한 sentencepiece 모델 불러오기
m = model.ModelProto()
m.ParseFromString(open('/content/MyDrive/MyDrive/프로젝트/한영데이터/mt5/spiece.model','rb').read())

4309802

In [35]:
# 기존 sentencepiece의 모델 크기
print(len(m.pieces)) # 250100

250100


In [36]:
# 새로운 pieces 만들기
new_pieces = [m.pieces[idx] for idx in tokens]
# new_pieces 개수 : 40426
print('new_pieces: ',len(new_pieces))

new_pieces:  40426


In [37]:
for i,p in enumerate(new_pieces):
    m.pieces[i].piece = p.piece
    m.pieces[i].score = p.score
    m.pieces[i].type = p.type

In [38]:
#from tqdm import trange
# drop rest
n = len(new_pieces)
for i in range(len(m.pieces)-n):
    m.pieces.pop(len(m.pieces)-1)

In [39]:
print(len(m.pieces))

40426


In [40]:
with open('/content/MyDrive/MyDrive/프로젝트/한영데이터/mt5/spiece.model', 'wb') as f:
    f.write(m.SerializeToString())

In [41]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer = T5Tokenizer('/content/MyDrive/MyDrive/프로젝트/한영데이터/mt5/spiece.model')

In [42]:
tokenizer

PreTrainedTokenizer(name_or_path='', vocab_size=40526, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_4